In [1]:
import pandas as pd
import numpy as np
import time
import requests
from tqdm import tqdm 

## Nutrition Facts

In [2]:
proxy_ip = "191.102.181.40"
proxy_port = "9584"
proxy_user = "CK3LWQ"
proxy_pass = "c0wg7U"

proxy_url = f"http://{proxy_user}:{proxy_pass}@{proxy_ip}:{proxy_port}"
proxies = {
    "http": proxy_url,
    "https": proxy_url,
}

api_key = 'O6EQ2mEKz27UlgihujBjt1RqJRc8ZteNiNLWBdQT'

In [3]:
input_file_epi_r = 'data/epi_r.csv'
df = pd.read_csv(input_file_epi_r)

In [4]:
for column in df.columns:
    if '/' in column:
        new_name_column = column.split('/')[0]
        df.rename(columns={column: new_name_column}, inplace=True)
    if ' or ' in column:
        new_name_column = column.split(' or ')[0]
        df.rename(columns={column: new_name_column}, inplace=True)

In [5]:
list_ingredients = []
session = requests.Session()
my_params = {
    "api_key": api_key,
    "query": ''
}
for column in tqdm(df.columns):
    my_params['query'] = column
    my_url = f'https://api.nal.usda.gov/fdc/v1/foods/search'
    response = session.get(url=my_url, proxies=proxies, params=my_params, timeout=10)
    json_tmp = response.json()
    if 'foods' in json_tmp:
        for food in json_tmp['foods']:
            if 'description' in food:
                if column.lower() == food['description'].lower():
                    list_ingredients.append(column)
                    break
list_ingredients = list(set(list_ingredients))

100%|██████████| 680/680 [09:44<00:00,  1.16it/s]


In [6]:
list_ingredients = pd.DataFrame(list_ingredients)
list_ingredients.to_csv("data/ingredients.csv", index=False)

In [7]:
columns_name_for_check = ['title', 'leftovers', 'california', 'dominican republic', 'protein', 'low cholesterol',
                         'pan-fry', 'fruit', 'stew', 'braise', 'milk', 'jam']
for column in columns_name_for_check:
    if column in list(df.columns):
        print(f"'{column}' in list(df.columns)")
    else:
        print(f"'{column}' not in list(df.columns)")

'title' in list(df.columns)
'leftovers' in list(df.columns)
'california' in list(df.columns)
'dominican republic' in list(df.columns)
'protein' in list(df.columns)
'low cholesterol' in list(df.columns)
'pan-fry' in list(df.columns)
'fruit' in list(df.columns)
'stew' in list(df.columns)
'braise' in list(df.columns)
'milk' in list(df.columns)
'jam' in list(df.columns)


In [8]:
ingredients = pd.read_csv(
    "data/ingredients.csv", header=None, names=["ingredient"])
ingredients = ingredients[ingredients['ingredient'] != "0"].reset_index(drop=True)
ingredients_list = ingredients['ingredient'].tolist()
print(ingredients_list)

['corn', 'butternut squash', 'egg nog', 'hamburger', 'rice', 'blueberry', 'bread', 'caraway', 'martini', 'onion', 'pasta', 'escarole', 'peanut butter', 'pizza', 'broccoli rabe', 'pepper', 'scotch', 'walnut', 'wild rice', 'ham', 'tofu', 'salmon', 'quinoa', 'cream cheese', 'maple syrup', 'marshmallow', 'meatball', 'cod', 'sausage', 'vinegar', 'octopus', 'couscous', 'orange juice', 'soy sauce', 'mushroom', 'apricot', 'caviar', 'fruit juice', 'jam', 'salsa', 'egg', 'blackberry', 'mozzarella', 'papaya', 'rosemary', 'waffle', 'basil', 'bulgur', 'pickles', 'pomegranate juice', 'saffron', 'ginger', 'curry', 'sorbet', 'cornmeal', 'guava', 'iced tea', 'peanut', 'shrimp', 'coconut', 'monterey jack', 'paprika', 'green bean', 'hummus', 'apple', 'halibut', 'milk', 'wasabi', 'granola', 'pistachio', 'radish', 'rum', 'raspberry', 'steak', 'brownie', 'lychee', 'tarragon', 'pie', 'thyme', 'lasagna', 'tortillas', 'cookies', 'swiss cheese', 'biscuit', 'broccoli', 'cottage cheese', 'gouda', 'mustard', 'rye'

In [9]:
session = requests.Session()
session.proxies.update(proxies)
nutrition_url = "https://api.nal.usda.gov/fdc/v1/foods/search"
nutrition_data = []

daily = pd.read_csv(
    "data/daily.tsv", sep="\t",
    converters={"Daily Value": lambda value: float(value.replace(",", ""))}
)

for ingredients_list in tqdm(ingredients_list, desc="Fetching nutrition data"):
    try:
        response = session.get(
            nutrition_url,
            params={"query": ingredients_list, "api_key": api_key, "pageSize": 3},
            timeout=10
        )
        if response.status_code != 200:
            print(f"Сервер вернул {response.status_code} для {ingredients_list}")
            continue

        nutrition_info = response.json()
        foods = nutrition_info.get("foods", [])
        if not foods:
            print(f"Нет данных для {ingredients_list}")
            continue

        # ищем подходящий продукт
        for food in foods:
            desc = food.get("description", "").lower()
            if ingredients_list.lower() not in desc:
                continue

            nutrients = food.get("foodNutrients", [])
            nutrient_dict = {"ingredient": ingredients_list}

            for nutrient in nutrients:
                name = nutrient.get("nutrientName", "").lower()
                value = nutrient.get("value")
                if value is None:
                    continue
                for _, row in daily.iterrows():
                    if row["Nutrient"].lower() in name:
                        try:
                            nutrient_dict[row["Nutrient"]] = round(value / row["Daily Value"] * 100)
                        except ZeroDivisionError:
                            nutrient_dict[row["Nutrient"]] = None
                        break

            nutrition_data.append(nutrient_dict)
            break  # нашли подходящий продукт, идём к следующему ингредиенту

    except Exception as e:
        print(f"Ошибка для {ingredients_list}: {e}")
        continue

Fetching nutrition data: 100%|██████████| 236/236 [02:36<00:00,  1.51it/s]


In [10]:
nutrition_df = pd.DataFrame(nutrition_data)
nutrition_df.to_csv("data/nutrition_facts.csv", index=False)

In [11]:
recipe_urls = []

for _, row in df.iterrows():
    try:
        title = row.get("title")
        rating = row.get("rating", None)

        if pd.notna(title):
            url = f"https://www.epicurious.com/recipes/food/views/{str(title).replace(' ', '-').replace(',', '').replace('\"', '').lower()}"
            recipe_urls.append({
                "title": str(title),
                "rating": rating,
                "url": url
            })
    except Exception as e:
        print(f"Ошибка при обработке строки: {e}")
        continue

In [12]:
recipe_urls_df = pd.DataFrame(recipe_urls)
recipe_urls_df.to_csv("data/similar_recipes.csv", index=False)